In [2]:
import numpy as np
import random
from copy import deepcopy

from utils import PriorityQueue, Node

### modeling

In [3]:
class RubikCube:
    def __init__(self, initial) -> None:
        self.initial = initial
        self.sides = dict(
            # face to face
            white=dict(up="red", down="orange", left="green", right="blue"), 
            yellow=dict(up="red", down="orange", left="blue", right="green"), 
            # face to face
            blue=dict(up="yellow", down="white", left="red", right="orange"), 
            green=dict(up="yellow", down="white", left="orange", right="red"),
            # face to face
            red=dict(up="yellow", down="white", left="green", right="blue"),  
            orange=dict(up="yellow", down="white", left="blue", right="green")
        )


    def goal_test(self, state: dict) -> bool:
        for side in state:
            center_color = side[0].upper()
            side_items = state[side].reshape(
                state[side].shape[0] * state[side].shape[1]
            )
            if any(map(lambda x : x[0] != center_color, side_items)):
                return False
        return True

    def actions(self, state: dict) -> list[tuple]:
        colors = list(state.keys())
        rotations = ["L", "R"]
        acts = []
        for color in colors:
            for rotate in rotations:
                acts.append((color, rotate))
        return acts
        

    def result(self, state, action) -> dict:
        new_state = deepcopy(state)
        self.rubik_rotate(new_state, *action)
        return new_state

    def heuristic(self, state) -> int:
        """num of incomplete surface """
        num_wrongs = 0
        for side in state:
            center_color = side[0].upper()
            side_items = state[side].reshape(
                state[side].shape[0] * state[side].shape[1]
            )
            num_wrongs += any(map(lambda x : x[0] != center_color, side_items))
        return num_wrongs
        
    def h(self, node) -> int:
        return self.heuristic(node.state)
    
    def path_cost(self, c, state1, action, state2):
        return c + 1
    
    @staticmethod
    def make_rubik():
        names = ['white', 'yellow', 'red', 'blue', 'green', 'orange']
        rubik = {}
        for name in names:
            arr = [f"{name[0].upper()}-{i}" for i in range(1, 3**2 + 1)]
            rubik[name] = np.array(arr, dtype=str).reshape(3, 3)
        return rubik
    
    def reverse_arr(self, arr):
        n = len(arr)
        copy = arr.copy()
        for i in range(n//2):
            copy[i], copy[n-i-1] = copy[n-i-1], copy[i]
        return copy
    
    def twod_array_rotate(self, array, rotate="L"):
        """rotate : 'L' or 'R' 
        array : np.array with shape (x, x)
        """
        shape = array.shape
        if len(shape) != 2 or shape[0] != shape[1]:
            raise f"array.sahpe : {array}, All axes must be the same"
        if rotate not in ["L", "R"]:
            raise "rotate must be 'L' or 'R' "
        
        surface = array
        surcopy = surface.copy()
        n = array.shape[0]
        for i in range(n):
            if rotate == "R":
                # Right rotate
                surface[i] = self.reverse_arr(surcopy[:, i])
            elif rotate == "L":
                # Left rotate
                surface[: , i] = self.reverse_arr(surcopy[i])

        return surface
    
    def advers_side(self, color, side):
        target_color = self.sides[color][side]
        for key, val in self.sides[target_color].items():
            if val == color:
                return key

    def get_side_row(self, array, side):
        arr = array.copy()
        if side == "up":
            return arr[0, :]
        if side == "down":
            return arr[2, :]
        if side == "left":
            return arr[:, 0]
        if side == "right":
            return arr[:, 2]

    def set_side_row(self, _2d_array, array, side: str):
        if side == "up":
            _2d_array[0, :] = array
        if side == "down":
            _2d_array[2, :] = array
        if side == "left":
            _2d_array[:, 0] = array
        if side == "right":
            _2d_array[:, 2] = array

    def rubik_rotate(self, rubik, color: str, rotate: str):
        """choose between 
        colors: 'white', 'yellow', 'red', 'blue', 'green', 'orange'
        rotate: 'L' or 'R' 
        """
        rubik[color] = self.twod_array_rotate(rubik[color], rotate)
        
        up = self.get_side_row(rubik[self.sides[color]["up"]], side=self.advers_side(color=color, side="up"))
        down = self.get_side_row(rubik[self.sides[color]["down"]], side=self.advers_side(color=color, side="down"))
        left = self.get_side_row(rubik[self.sides[color]["left"]], side=self.advers_side(color=color, side="left"))
        right = self.get_side_row(rubik[self.sides[color]["right"]], side=self.advers_side(color=color, side="right"))

        if rotate == "L":
            self.set_side_row(rubik[self.sides[color]["up"]], array=left, 
                              side=self.advers_side(color=color, side="up"))
            self.set_side_row(rubik[self.sides[color]["right"]], array=up, 
                              side=self.advers_side(color=color, side="right"))
            self.set_side_row(rubik[self.sides[color]["down"]], array=right, 
                              side=self.advers_side(color=color, side="down"))
            self.set_side_row(rubik[self.sides[color]["left"]], array=down, 
                              side=self.advers_side(color=color, side="left"))
            
        elif rotate == "R":
            self.set_side_row(rubik[self.sides[color]["up"]], array=right, 
                              side=self.advers_side(color=color, side="up"))
            self.set_side_row(rubik[self.sides[color]["right"]], array=down, 
                              side=self.advers_side(color=color, side="right"))
            self.set_side_row(rubik[self.sides[color]["down"]], array=left, 
                              side=self.advers_side(color=color, side="down"))
            self.set_side_row(rubik[self.sides[color]["left"]], array=up, 
                              side=self.advers_side(color=color, side="left"))
    
    def random_actions(self, n=10):
        colors = ['white', 'yellow', 'red', 'blue', 'green', 'orange']
        rotations = ["L", "R"]
        shuffled_movements = []
        for _ in range(n):
            color = random.choice(colors)
            rotate = random.choice(rotations)
            shuffled_movements.append((color, rotate))
        return shuffled_movements
    
    def shuffle_rubik(self, state, n=10):
        random_actions = self.random_actions(n)
        for action in random_actions:
            self.rubik_rotate(state, *action)
        return random_actions

    def display(self, rubik):
        for key, val in rubik.items():
            print(key)
            print(val)

    

In [4]:
def aStar(problem, wight=1.0, display=False, graph_search=True) -> Node | None:
    h = problem.h 
    frontier = PriorityQueue()
    explored = set()
    node = Node(problem.initial)
    frontier.push(node, wight*h(node) + node.path_cost)
    c = 0
    while not frontier.isEmpty():
        node: Node = frontier.pop()
        if graph_search:
            explored.add(str(node.state))
        
        if problem.goal_test(node.state):
            if display:
                print("frontier count:", frontier.count)
                print("expanded nodes:", c)
            return node
        
        for child in node.expand(problem):
            if str(child.state) not in explored:
                frontier.push(child, wight*h(node) + node.path_cost)
        c += 1
    return None

In [6]:
## initial the problem
problem = RubikCube(initial=RubikCube.make_rubik())
random_actions = problem.shuffle_rubik(problem.initial, 5)

"{:,}".format(12**5) # search space

'248,832'

In [7]:
## A* search
res = aStar(problem, display=True)
path = res.path()

print("path length:", len(path))
for i, node in enumerate(path):
    print(f"\nstep {i}")
    print("action:", node.action)
    problem.display(node.state)

frontier count: 98382
expanded nodes: 9194
path length: 6

step 0
action: None
white
[['W-9' 'W-8' 'Y-1']
 ['W-6' 'W-5' 'Y-4']
 ['B-9' 'R-6' 'B-3']]
yellow
[['R-1' 'Y-2' 'O-3']
 ['W-2' 'Y-5' 'O-6']
 ['Y-9' 'Y-8' 'O-9']]
red
[['Y-3' 'R-2' 'O-1']
 ['Y-6' 'R-5' 'O-4']
 ['W-7' 'B-6' 'G-1']]
blue
[['W-3' 'O-8' 'O-7']
 ['G-4' 'B-5' 'B-4']
 ['G-7' 'B-2' 'B-1']]
green
[['G-3' 'G-6' 'G-9']
 ['G-2' 'G-5' 'G-8']
 ['B-7' 'B-8' 'Y-7']]
orange
[['R-7' 'O-2' 'W-1']
 ['R-4' 'O-5' 'W-4']
 ['R-3' 'R-8' 'R-9']]

step 1
action: ('blue', 'L')
white
[['W-9' 'W-8' 'R-7']
 ['W-6' 'W-5' 'R-4']
 ['B-9' 'R-6' 'R-3']]
yellow
[['O-1' 'Y-2' 'O-3']
 ['O-4' 'Y-5' 'O-6']
 ['G-1' 'Y-8' 'O-9']]
red
[['Y-3' 'R-2' 'Y-1']
 ['Y-6' 'R-5' 'Y-4']
 ['W-7' 'B-6' 'B-3']]
blue
[['O-7' 'B-4' 'B-1']
 ['O-8' 'B-5' 'B-2']
 ['W-3' 'G-4' 'G-7']]
green
[['G-3' 'G-6' 'G-9']
 ['G-2' 'G-5' 'G-8']
 ['B-7' 'B-8' 'Y-7']]
orange
[['R-1' 'O-2' 'W-1']
 ['W-2' 'O-5' 'W-4']
 ['Y-9' 'R-8' 'R-9']]

step 2
action: ('white', 'L')
white
[['R-7' 'R-4' 'R